## 동시에 여러 DB 수행시키기
1. 동일 시점에 스키마정보를 수집해야 할때 (SchemaCollectMulti)
2. SQL생성시 여러 DB를 동시에 Insert 시키고 싶을 때(※시간 단축됨) (SQLCreateMulti)
3.여러 DB에 동시에 SQL을 수행시켜야 할때 (ExecuteMulti)

In [1]:
from threading import Thread
import pyDataProfiling

### 1. 동일 시점에 스키마정보를 수집해야 할때 (SchemaCollectMulti)

In [ ]:
### 스레드에서 동시 처리할 수 있도록 별도 함수 생성
def SchemaCollectMulti(DB번호):
    DataProfiler = pyDataProfiling.DataProfiling()

    ### 리포지터리DB 지정
    DataProfiler.RepositoryDBSetting(dbtype='Tibero', ip='127.0.0.1', port=1521, dbname='trcl', userid='user', password='password')

    ### 리포지터리DB 접속
    DataProfiler.RepositoryDBConnect()

    ### 프로파일링 대상 DB 스키마 정보 로드
    DataProfiler.ProfilingDBLoad()
    DataProfiler.ProfilingDBConnect(DB번호=DB번호)
    
    ### 스키마정보 수집
    DataProfiler.DictCollector.Collect(return_type='Repository', ignore=True)

In [ ]:
### 동시에 수행
threads = []
DB목록 = [1,2,3,4,5]
for DB번호 in DB목록:
    t = Thread(target=SchemaCollectMulti, args=(DB번호, ))
    threads.append(t)
    t.start()

### 2. SQL생성시 여러 DB를 동시에 Insert 시키고 싶을 때(※시간 단축됨) (SQLCreateMulti)

In [ ]:
### 스레드에서 동시 처리할 수 있도록 별도 함수 생성
def SQLCreateMulti(DB번호):
    DataProfiler = pyDataProfiling.DataProfiling()

    ### 리포지터리DB 지정
    DataProfiler.RepositoryDBSetting(dbtype='Tibero', ip='127.0.0.1', port=1521, dbname='trcl', userid='user', password='password')

    ### 리포지터리DB 접속
    DataProfiler.RepositoryDBConnect()

    ### 프로파일링 대상 DB 스키마 정보 로드
    ### 딕셔너리 조회
    DataProfiler.SQLCreator.DictionaryListSelect(DB번호=DB번호, schemaList=['DTWAREADM'])
    
    ### 컬럼기초통계치 수집 SQL 생성
    DataProfiler.SQLCreator.ColumnProfingSQLCreate(estimate='sample', rowcount=100)
    
    ### 샘플데이터 수집 SQL 생성
    DataProfiler.SQLCreator.SampleProfingSQLCreate(estimate='sample', rowcount=100)

In [ ]:
### 동시에 수행
threads = []
DB목록 = [1,2,3,4,5]
for DB번호 in DB목록:
    t = Thread(target=SQLCreateMulti, args=(DB번호, ))
    threads.append(t)
    t.start()

### 3.여러 DB에 동시에 SQL을 수행시켜야 할때 (ExecuteMulti)

In [2]:
### 스레드에서 동시 처리할 수 있도록 별도 함수 생성
def ExecuteMulti(DB번호, degree):
    DataProfiler = pyDataProfiling.DataProfiling()

    ### 리포지터리DB 지정
    DataProfiler.RepositoryDBSetting(dbtype='Tibero', ip='127.0.0.1', port=1521, dbname='trcl', userid='user', password='password')

    ### 리포지터리DB 접속
    DataProfiler.RepositoryDBConnect()

    ### 로드
    DataProfiler.ProfilingDBLoad()
    
    ### 접속
    DataProfiler.ProfilingDBConnect(DB번호=DB번호)    
    
    DataProfiler.Executer.execute(degree=degree, duplicate='merge')

In [ ]:
### 동시에 수행
threads = []
DB목록 = [[1, 1],[2,2],[3,1],[4, 2],[5, 1]]
 
for DB번호, degree in DB목록:
    t = Thread(target=ExecuteMulti, args=(DB번호, degree, ))
    threads.append(t)
    t.start()
    
for t in threads:
    t.join()